In [ ]:
#| default_exp verification

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dataclasses import dataclass
from typing import Any

from pymoq.mocking.functions import FunctionMock

from fastcore.basics import patch_to

# Verification
> Verify that the mock was called the way you intended

In [ ]:
from pymoq.mocking import objects
from typing import Protocol

Mock = objects.Mock

In [ ]:
class IWeb(Protocol):
    def get(a: int, b:str, c:float|None=None):
        ...

In [ ]:
m = Mock(IWeb)

m.get(1,"2")

m.get._calls

[((1, '2'), {'c': None})]

In [ ]:
from pymoq.signature_validators import signature_validator_from_arguments
from pymoq.mocking.functions import add_self_parameter

In [ ]:
sign_val = signature_validator_from_arguments(['a', 'b', 'c'], 1, "2", c=lambda _: True)

In [ ]:
for args, kwargs in m.get._calls:
    kwargs = m.get.fill_up_arg_list(args, kwargs)
    print(sign_val.is_valid(*args, **kwargs))

True


In [ ]:
@dataclass
class VerifiedCalls:
    calls: list[tuple[list[Any], dict[str, Any]]]
    
    @property
    def successful(self): return len(self.calls)

In [ ]:
@patch_to(FunctionMock)
def verify(self, *args, **kwargs) -> VerifiedCalls:
    kwargs = self.fill_up_arg_list(args, kwargs)
    sign_val = signature_validator_from_arguments(self._argument_names, *args, **kwargs)
    calls = []
    
    for call_args, call_kwargs in self._calls:
        call_kwargs = self.fill_up_arg_list(call_args, call_kwargs)
        if sign_val.is_valid(*call_args, **call_kwargs):
            calls.append((call_args, call_kwargs))
    return VerifiedCalls(calls)

In [ ]:
m = Mock(IWeb)

m.get(1,"2")
m.get(2,"2")

m.get.verify(lambda x: isinstance(x, int),"2")

VerifiedCalls(calls=[((1, '2'), {'c': None}), ((2, '2'), {'c': None})])